In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import plotly.graph_objects as go

# Define the logistic function
def logistic_function(x, capacity, growth_rate, x0):
    """The logistic function."""
    return capacity / (1 + np.exp(-growth_rate * (x - x0)))

# Load the data
file_path = r"C:\Users\daves\OneDrive\Pessoal\Acadêmico\Mestrado\Dissertação - Execução\Análises\Dados do Censo - não usar na regressão\PIA individuais.xlsx"
df = pd.read_excel(file_path, sheet_name="Cuiabá")

# Filter out zeros or negative values
df = df[df['Totalammount'] > 0].copy()

# Define municipalities and their colors/line styles
municipality_styles = {
    "Acorizal": {'color': 'green', 'line_style': '--'},
    "Barão de Melgaço": {'color': 'green', 'line_style': '-'},
    "Campo Verde": {'color': 'blue', 'line_style': '--'},
    "Chapada dos Guimarães": {'color': 'blue', 'line_style': '-'},
    "Cuiabá": {'color': 'red', 'line_style': '--'},
    "Jangada": {'color': 'black', 'line_style': '-'},
    "Nobres": {'color': 'black', 'line_style': '--'},
    "Nossa Senhora do Livramento": {'color': 'yellow', 'line_style': '-'},
    "Nova Brasilândia": {'color': 'yellow', 'line_style': '--'},
    "Planalto da Serra": {'color': 'orange', 'line_style': '-'},
    "Poconé": {'color': 'purple', 'line_style': '--'},
    "Rosário Oeste": {'color': 'gray', 'line_style': '-'},
    "Santo Antônio do Leverger": {'color': 'gray', 'line_style': '--'},
    "Várzea Grande": {'color': 'brown', 'line_style': '-'}
}

# ==============================================
# STATIC PLOT (matplotlib)
# ==============================================
plt.figure(figsize=(24, 16))

# Set global font size to 16
plt.rcParams.update({
    'font.size': 16,
    'axes.titlesize': 16,
    'axes.labelsize': 16,
    'xtick.labelsize': 16,
    'ytick.labelsize': 16,
    'legend.fontsize': 16
})

# For each municipality, perform logistic fit and plot
for mun, styles in municipality_styles.items():
    mun_df = df[df['Municipios'] == mun]
    if len(mun_df) >= 3:  # Need at least 3 points for a logistic fit
        try:
            years = np.array(mun_df['Year'])
            amounts = np.array(mun_df['Totalammount'])
            
            # Initial guesses for parameters
            p0 = [max(amounts) * 1.5, 0.5, np.median(years)]
            bounds = ([max(amounts), 0.01, min(years)], [max(amounts) * 10, 5, max(years)])
            
            # Fit logistic curve
            popt_logistic, _ = curve_fit(logistic_function, years, amounts, p0=p0, bounds=bounds, maxfev=10000)
            
            # Generate predictions
            years_pred = np.linspace(min(years) - 2, max(years) + 2, 100)
            pred_logistic = logistic_function(years_pred, *popt_logistic)
            
            # Plot the fitted line with thicker lines
            plt.plot(years_pred, pred_logistic, 
                     color=styles['color'], 
                     linestyle=styles['line_style'], 
                     linewidth=3.0,
                     label=mun)
            
        except RuntimeError as e:
            print(f"Could not fit logistic curve for {mun}: {str(e)}")
        except Exception as e:
            print(f"An error occurred during logistic fit for {mun}: {str(e)}")

# Customize the plot
plt.xlabel('Year', fontsize=16)
plt.ylabel('')  # Empty string removes y-axis label
plt.yticks(rotation=45)
plt.grid(False)

# Legend settings with larger font
legend = plt.legend(
    fontsize=16,
    ncol=6,
    bbox_to_anchor=(0.5, -0.15),
    loc='upper center',
    frameon=True,
    framealpha=0.8,
    edgecolor='black',
    borderpad=1.2
)

# Adjust layout to make room for the legend
plt.tight_layout(rect=[0, 0.15, 1, 0.95])

# Save the static plot with 1200 DPI
static_output_path = r"C:/Users/daves/OneDrive/Pessoal/Acadêmico/Mestrado/Dissertação - Execução/Dissertação/Quadros e gráficos/Graph8_logistic_static.jpg"
plt.savefig(static_output_path, dpi=1200, bbox_inches='tight', facecolor='white')
plt.close()  # Close the figure to free memory

C:\Users\daves\AppData\Local\Temp\ipykernel_26296\1888920079.py:65: OptimizeWarning:

Covariance of the parameters could not be estimated



In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import plotly.graph_objects as go

# Define the logistic function
def logistic_function(x, capacity, growth_rate, x0):
    """The logistic function."""
    return capacity / (1 + np.exp(-growth_rate * (x - x0)))

# Load the data
file_path = r"C:\Users\daves\OneDrive\Pessoal\Acadêmico\Mestrado\Dissertação - Execução\Análises\Dados do Censo - não usar na regressão\PIA individuais.xlsx"
df = pd.read_excel(file_path, sheet_name="Cuiabá")

# Filter out zeros or negative values
df = df[df['Totalammount'] > 0].copy()

# Define municipalities and their colors/line styles
municipality_styles = {
    "Acorizal": {'color': 'green', 'line_style': '--'},
    "Barão de Melgaço": {'color': 'green', 'line_style': '-'},
    "Campo Verde": {'color': 'blue', 'line_style': '--'},
    "Chapada dos Guimarães": {'color': 'blue', 'line_style': '-'},
    "Cuiabá": {'color': 'red', 'line_style': '--'},
    "Jangada": {'color': 'black', 'line_style': '-'},
    "Nobres": {'color': 'black', 'line_style': '--'},
    "Nossa Senhora do Livramento": {'color': 'yellow', 'line_style': '-'},
    "Nova Brasilândia": {'color': 'yellow', 'line_style': '--'},
    "Planalto da Serra": {'color': 'orange', 'line_style': '-'},
    "Poconé": {'color': 'purple', 'line_style': '--'},
    "Rosário Oeste": {'color': 'gray', 'line_style': '-'},
    "Santo Antônio do Leverger": {'color': 'gray', 'line_style': '--'},
    "Várzea Grande": {'color': 'brown', 'line_style': '-'}
}

# ==============================================
# INTERACTIVE PLOT (plotly) - MAIN FOCUS
# ==============================================
fig = go.Figure()

# For each municipality, add logistic fit to interactive plot
for mun, styles in municipality_styles.items():
    mun_df = df[df['Municipios'] == mun]
    if len(mun_df) >= 3:
        try:
            years = np.array(mun_df['Year'])
            amounts = np.array(mun_df['Totalammount'])
            
            # Fit logistic curve
            p0 = [max(amounts) * 1.5, 0.5, np.median(years)]
            bounds = ([max(amounts), 0.01, min(years)], [max(amounts) * 10, 5, max(years)])
            popt_logistic, _ = curve_fit(logistic_function, years, amounts, p0=p0, bounds=bounds, maxfev=10000)
            
            # Generate predictions
            years_pred = np.linspace(min(years) - 2, max(years) + 2, 100)
            pred_logistic = logistic_function(years_pred, *popt_logistic)
            
            # Add trace to interactive plot
            fig.add_trace(go.Scatter(
                x=years_pred, 
                y=pred_logistic,
                mode='lines',
                line=dict(
                    color=styles['color'],
                    width=3,
                    dash='dash' if styles['line_style'] == '--' else 'solid'
                ),
                name=mun,
                visible=True,
                hovertemplate='Year: %{x}<br>Total Amount: %{y:.2f}<extra></extra>'
            ))
        except Exception as e:
            print(f"Error creating interactive plot for {mun}: {str(e)}")

# Create dropdown buttons to toggle visibility
buttons = []
buttons.append(dict(
    label="All",
    method="update",
    args=[{"visible": [True] * len(fig.data)},
          {"title": "Total Amount Logistic Fits for All Municipalities"}]
))

for i, mun in enumerate(municipality_styles.keys()):
    visible = [False] * len(fig.data)
    for j, trace in enumerate(fig.data):
        if trace.name == mun:
            visible[j] = True
    buttons.append(dict(
        label=mun,
        method="update",
        args=[{"visible": visible},
              {"title": f"Total Amount Logistic Fit for {mun}"}]
    ))

# Update layout with dropdown BELOW TITLE
fig.update_layout(
    title=dict(
        text='Total Amount Logistic Fits for Cuiabá Immediate Region',
        y=0.95,
        x=0.5,
        xanchor='center',
        yanchor='top'
    ),
    updatemenus=[dict(
        buttons=buttons,
        direction="down",
        pad={"r": 10, "t": 10, "b": 10},
        showactive=True,
        x=0.5,
        xanchor="center",
        y=0.88,  # Positioned just below title
        yanchor="top"
    )],
    xaxis_title='Year',
    yaxis_title='Total Amount',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.3,
        xanchor="center",
        x=0.5,
        font=dict(size=12)
    ),
    font=dict(size=14),
    plot_bgcolor='white',
    height=800,
    margin=dict(t=100, b=150, l=50, r=50)  # Adjusted margins
)

# Save the interactive plot
interactive_output_path = r"C:/Users/daves/OneDrive/Pessoal/Acadêmico/Mestrado/Dissertação - Execução/Dissertação/Quadros e gráficos/WAP(Cuiabá Immediate Region).html"
fig.write_html(interactive_output_path)

print(f"Interactive plot saved to: {interactive_output_path}")

# Show the interactive plot
fig.show()

Interactive plot saved to: C:/Users/daves/OneDrive/Pessoal/Acadêmico/Mestrado/Dissertação - Execução/Dissertação/Quadros e gráficos/WAP(Cuiabá Immediate Region).html


C:\Users\daves\AppData\Local\Temp\ipykernel_26296\2037514275.py:53: OptimizeWarning:

Covariance of the parameters could not be estimated

